## Feature Selection

### Outline <a name = 'outline'></a>
* [1. mutual_info_regression](#mutual_info_regression)
* [2. mutual_info_classif](#mutual_info_classif)
* [3. permutation importance](#permutation_importance)
* [4. BoostARoota](#BoostARoota)
* [5. shap](#shap)
* [6. SelectKBest](#SelectKBest)
* [7. SelectPercentile](#SelectPercentile)
* [8. SelectFromModel](#SelectFromModel)
* [9. RFE](#RFE)
* [10. RFECV](#RFECV)
* [11. SequentialFeatureSelector](#SequentialFeatureSelector)

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston

from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
from sklearn.inspection import permutation_importance
from boostaroota import BoostARoota
import shap
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile, SelectFromModel, RFE, RFECV 
from mlxtend.feature_selection import SequentialFeatureSelector

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
load_boston().keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [3]:
print(load_boston()['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [4]:
X, y = load_boston(return_X_y=True)

In [5]:
features = load_boston()['feature_names']
df = pd.DataFrame(X, columns=features)
df['MEDV'] = y
df.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.9,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6


In [6]:
discrete_feature_names = []
continuous_feature_names = []
for feature in features:
    if df[feature].nunique() < 20:
        discrete_feature_names.append(feature)
    else:
        continuous_feature_names.append(feature)
        
print(f'discrete_feature_names: {discrete_feature_names}') 
print(f'continuous_feature_names: {continuous_feature_names}')

discrete_feature_names: ['CHAS', 'RAD']
continuous_feature_names: ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']


In [7]:
print(f'X.shape {X.shape}')
print(f'y.shape {y.shape}')

X.shape (506, 13)
y.shape (506,)


### 1. mutual_info_regression<a name = 'mutual_info_regression'></a>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html

In [8]:
score = mutual_info_regression(X, y)
t = np.quantile(score, 0.8)
score[score > t]

array([0.46460057, 0.53025449, 0.67119325])

In [9]:
features[score > t]

array(['INDUS', 'RM', 'LSTAT'], dtype='<U7')

[back to outline](#outline)

### 2. mutual_info_classif<a name = 'mutual_info_classif'></a>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif

In [10]:
q = np.quantile(y, 0.8)
y_class = np.zeros(y.shape)
y_class[y > q] = 1

In [11]:
score = mutual_info_classif(X, y_class)
t = np.quantile(score, 0.8)
score[score > t]

array([0.25167465, 0.26009002, 0.28913441])

In [12]:
features[score > t]

array(['INDUS', 'NOX', 'RM'], dtype='<U7')

[back to outline](#outline)

### 3. permutation importance<a name = 'permutation_importance'></a>
https://scikit-learn.org/stable/modules/permutation_importance.html

In [13]:
def get_permutation_importance(model):
    importance = permutation_importance(
        model, X_test, y_test, scoring="roc_auc", n_repeats=30, random_state=42)

    importance_scores = pd.DataFrame(
        {"features": features, "importance-mean": importance.importances_mean}
    ).sort_values(by="importance-mean", ascending=False)

    return importance_scores

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_class, test_size=0.2, random_state=16)

LogisticRegression

In [15]:
model_lr = LogisticRegression(penalty="l2", C=0.1).fit(X_scaled, y_class)
importance_scores = get_permutation_importance(model_lr)
importance_scores

,features,importance-mean
5,RM,7.529762e-02
12,LSTAT,2.727273e-02
2,INDUS,3.733766e-03
4,NOX,8.387446e-04
9,TAX,7.305195e-04
7,DIS,5.952381e-04
10,PTRATIO,8.116883e-05
3,CHAS,5.411255e-05
0,CRIM,0.000000e+00
6,AGE,0.000000e+00


#### XGBClassifier

In [16]:
model_xgb = XGBClassifier(n_estimators=72).fit(X_scaled, y_class)
importance_scores = get_permutation_importance(model_xgb)
importance_scores

,features,importance-mean
5,RM,5.473485e-02
12,LSTAT,1.685606e-02
9,TAX,3.246753e-04
10,PTRATIO,1.082251e-04
6,AGE,3.700743e-18
0,CRIM,0.000000e+00
1,ZN,0.000000e+00
2,INDUS,0.000000e+00
3,CHAS,0.000000e+00
4,NOX,0.000000e+00


[back to outline](#outline)

### 4. BoostARoota<a name = 'BoostARoota'></a>
https://github.com/chasedehan/BoostARoota

In [17]:
br = BoostARoota(metric='logloss', silent=True)

X_bar = pd.DataFrame(X_scaled, columns=features)
br.fit(X_bar, y)

[22:41:31] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:41:31] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:41:31] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [18]:
br.keep_vars_.tolist()

['CRIM',
 'ZN',
 'INDUS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

In [19]:
X_bar = br.transform(X_bar)
X_bar.head(2)

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.419782,0.284830,-1.287909,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439


[back to outline](#outline)

### 5. shap<a name = 'shap'></a>
https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/model_agnostic/Iris%20classification%20with%20scikit-learn.html

In [20]:
X_train_shap = X_train[np.random.choice(X_train.shape[0], 30), :]

In [21]:
def get_shap_importance(model):
    model.fit(X_train, y_train)
    explainer = shap.KernelExplainer(model.predict_proba, X_train_shap)
    shap_values = explainer.shap_values(X_test)
    
    importance = abs(shap_values[0].mean(axis=0))

    importance_scores = pd.DataFrame(
        {"features": features, "importance-mean": importance}
    ).sort_values(by="importance-mean", ascending=False)

    return importance_scores

#### LogisticRegression

In [22]:
importance_scores = get_shap_importance(model_lr)
importance_scores

,features,importance-mean
12,LSTAT,0.030980
5,RM,0.011208
8,RAD,0.006742
7,DIS,0.006017
1,ZN,0.003883
10,PTRATIO,0.002656
9,TAX,0.002282
4,NOX,0.001703
3,CHAS,0.001504
2,INDUS,0.000945


#### XGBClassifier

In [23]:
importance_scores = get_shap_importance(model_xgb)
importance_scores

,features,importance-mean
12,LSTAT,0.037637
9,TAX,0.016209
2,INDUS,0.012767
5,RM,0.011847
10,PTRATIO,0.004275
7,DIS,0.003137
4,NOX,0.001296
6,AGE,0.000887
8,RAD,0.000833
1,ZN,0.000464


#### KNeighborsClassifier

In [24]:
model_knn = KNeighborsClassifier()
importance_scores = get_shap_importance(model_knn)
importance_scores

,features,importance-mean
12,LSTAT,0.017458
10,PTRATIO,0.007201
9,TAX,0.006763
1,ZN,0.004796
3,CHAS,0.004488
7,DIS,0.004260
5,RM,0.004186
4,NOX,0.003458
8,RAD,0.003006
6,AGE,0.002938


[back to outline](#outline)

### 6. SelectKBest<a name = 'SelectKBest'></a>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest

In [25]:
X_scaled_pos = X_scaled - X_scaled.min(axis=0) #SelectKBest takes only positive X
kbest = SelectKBest(chi2, k=6).fit(X_scaled_pos, y_class)

In [26]:
kbest.get_params()

{'k': 6,
 'score_func': <function sklearn.feature_selection._univariate_selection.chi2(X, y)>}

In [27]:
features[kbest.get_support()]

array(['ZN', 'INDUS', 'RM', 'TAX', 'PTRATIO', 'LSTAT'], dtype='<U7')

In [28]:
importance_scores = pd.DataFrame(
        {"features": features, 
         "importance-mean": kbest.scores_,
         "p-values": kbest.pvalues_}
    ).sort_values(by="importance-mean", ascending=False)

importance_scores

,features,importance-mean,p-values
1,ZN,144.603390,2.622228e-33
12,LSTAT,84.821652,3.265289e-20
5,RM,54.461649,1.585113e-13
2,INDUS,53.544751,2.527689e-13
10,PTRATIO,38.645474,5.082096e-10
9,TAX,36.285075,1.704643e-09
0,CRIM,36.201050,1.779746e-09
4,NOX,26.525550,2.600756e-07
8,RAD,25.623808,4.148889e-07
3,CHAS,17.717504,2.562577e-05


[back to outline](#outline)

### 7. SelectPercentile<a name = 'SelectPercentile'></a>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile

In [29]:
percentile = SelectPercentile(chi2, percentile=20).fit(X_scaled_pos, y_class)

In [30]:
percentile.get_params()

{'percentile': 20,
 'score_func': <function sklearn.feature_selection._univariate_selection.chi2(X, y)>}

In [31]:
features[percentile.get_support()]

array(['ZN', 'RM', 'LSTAT'], dtype='<U7')

[back to outline](#outline)

### 8. SelectFromModel<a name = 'SelectFromModel'></a>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel

#### LogisticRegression

In [32]:
selector_lr = SelectFromModel(estimator=LogisticRegression(), threshold='1.25*mean').fit(X_scaled, y_class)
score = abs(selector_lr.estimator_.coef_)

In [33]:
selector_lr.threshold_

0.9441446541144094

In [34]:
features[selector_lr.get_support()]

array(['RM', 'RAD', 'LSTAT'], dtype='<U7')

In [35]:
importance_scores = pd.DataFrame(
        {"features": features, 
         "importance": abs(selector_lr.estimator_.coef_.flatten())}
    ).sort_values(by="importance", ascending=False)

importance_scores

,features,importance
12,LSTAT,2.527725
5,RM,1.656581
8,RAD,1.565058
7,DIS,0.925706
10,PTRATIO,0.869635
2,INDUS,0.821966
9,TAX,0.785063
1,ZN,0.269861
4,NOX,0.260992
6,AGE,0.060372


#### XGBClassifier

In [36]:
selector_xgb = SelectFromModel(estimator=XGBClassifier(
    max_depth=5, random_state=40, n_estimators=72
), threshold='1.25*mean').fit(X_scaled, y_class)
score = abs(selector_xgb.estimator_.feature_importances_)

In [37]:
importance_scores = pd.DataFrame(
        {"features": features, 
         "importance": abs(selector_xgb.estimator_.feature_importances_.flatten())}
    ).sort_values(by="importance", ascending=False)

importance_scores

,features,importance
5,RM,0.434495
12,LSTAT,0.136634
10,PTRATIO,0.130562
7,DIS,0.057019
9,TAX,0.050233
8,RAD,0.047855
2,INDUS,0.033684
0,CRIM,0.027676
11,B,0.024974
6,AGE,0.022862


[back to outline](#outline)

### 9. RFE<a name = 'RFE'></a>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE

#### LogisticRegression

In [38]:
selector_rfe = RFE(estimator=LogisticRegression(), n_features_to_select=3, step=1).fit(X_scaled, y_class)

In [39]:
features[selector_rfe.get_support()]

array(['RM', 'PTRATIO', 'LSTAT'], dtype='<U7')

In [40]:
importance_scores = pd.DataFrame(
        {"features": features, 
         "importance": selector_rfe.ranking_}
    ).sort_values(by="importance", ascending=True)

importance_scores

,features,importance
5,RM,1
10,PTRATIO,1
12,LSTAT,1
8,RAD,2
2,INDUS,3
7,DIS,4
9,TAX,5
1,ZN,6
4,NOX,7
6,AGE,8


#### XGBClassifier

In [41]:
selector_rfe_xgb = RFE(estimator=XGBClassifier(
    max_depth=5, random_state=40, n_estimators=72
), n_features_to_select=3, step=1).fit(X_scaled, y_class)

In [42]:
importance_scores = pd.DataFrame(
        {"features": features, 
         "importance": selector_rfe_xgb.ranking_}
    ).sort_values(by="importance", ascending=True)

importance_scores

,features,importance
5,RM,1
10,PTRATIO,1
12,LSTAT,1
7,DIS,2
9,TAX,3
2,INDUS,4
0,CRIM,5
8,RAD,6
11,B,7
6,AGE,8


[back to outline](#outline)

### 10. RFECV<a name = 'RFECV'></a>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV

#### LogisticRegression

In [43]:
selector_rfecv = RFECV(estimator=LogisticRegression(), step=1, cv=5).fit(X_scaled, y_class)

In [44]:
importance_scores = pd.DataFrame(
        {"features": features, 
         "importance": selector_rfecv.ranking_}
    ).sort_values(by="importance", ascending=True)

importance_scores

,features,importance
2,INDUS,1
5,RM,1
7,DIS,1
8,RAD,1
10,PTRATIO,1
12,LSTAT,1
9,TAX,2
1,ZN,3
4,NOX,4
6,AGE,5


#### XGBClassifier

In [45]:
selector_rfecv_xgb = RFECV(estimator=XGBClassifier(
    max_depth=5, random_state=40, n_estimators=72
), step=1, cv=5).fit(X_scaled, y_class)

In [46]:
importance_scores = pd.DataFrame(
        {"features": features, 
         "importance": selector_rfecv_xgb.ranking_}
    ).sort_values(by="importance", ascending=True)

importance_scores

,features,importance
5,RM,1
7,DIS,1
9,TAX,1
10,PTRATIO,1
12,LSTAT,1
2,INDUS,2
0,CRIM,3
8,RAD,4
11,B,5
6,AGE,6


[back to outline](#outline)

### 11. SequentialFeatureSelector<a name = 'SequentialFeatureSelector'></a>
http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#example-5-sequential-feature-selection-for-regression

#### LogisticRegression

In [47]:
xgb = XGBClassifier(max_depth=5, random_state=42, n_estimators=72)
sfs1 = SequentialFeatureSelector(
    LogisticRegression(), 
           k_features=4, 
           forward=True, 
           floating=True, 
           verbose=0,
           scoring='accuracy',
           cv=4)

sfs1 = sfs1.fit(X_scaled, y_class, custom_feature_names=features)

In [48]:
sfs1.subsets_

{1: {'feature_idx': (5,),
  'cv_scores': array([0.93700787, 0.92125984, 0.92063492, 0.88095238]),
  'avg_score': 0.9149637545306837,
  'feature_names': ('RM',)},
 2: {'feature_idx': (5, 12),
  'cv_scores': array([0.94488189, 0.94488189, 0.93650794, 0.92857143]),
  'avg_score': 0.938710786151731,
  'feature_names': ('RM', 'LSTAT')},
 3: {'feature_idx': (5, 10, 12),
  'cv_scores': array([0.92913386, 0.96062992, 0.93650794, 0.93650794]),
  'avg_score': 0.940694913135858,
  'feature_names': ('RM', 'PTRATIO', 'LSTAT')},
 4: {'feature_idx': (5, 6, 10, 12),
  'cv_scores': array([0.93700787, 0.95275591, 0.95238095, 0.92857143]),
  'avg_score': 0.9426790401199849,
  'feature_names': ('RM', 'AGE', 'PTRATIO', 'LSTAT')}}

In [49]:
sfs1.k_feature_names_

('RM', 'AGE', 'PTRATIO', 'LSTAT')

In [50]:
sfs1.k_score_

0.9426790401199849

#### XGBClassifier

In [51]:
xgb = XGBClassifier(max_depth=5, random_state=42, n_estimators=72)
sfs1 = SequentialFeatureSelector(
    xgb, 
           k_features=4, 
           forward=True, 
           floating=True, 
           verbose=0,
           scoring='accuracy',
           cv=4)

sfs1 = sfs1.fit(X_scaled, y_class, custom_feature_names=features)

In [52]:
sfs1.subsets_

{1: {'feature_idx': (5,),
  'cv_scores': array([0.91338583, 0.90551181, 0.8968254 , 0.84920635]),
  'avg_score': 0.8912323459567554,
  'feature_names': ('RM',)},
 2: {'feature_idx': (5, 12),
  'cv_scores': array([0.92125984, 0.92125984, 0.92857143, 0.94444444]),
  'avg_score': 0.9288838895138107,
  'feature_names': ('RM', 'LSTAT')},
 3: {'feature_idx': (4, 5, 12),
  'cv_scores': array([0.93700787, 0.92913386, 0.91269841, 0.92857143]),
  'avg_score': 0.9268528933883264,
  'feature_names': ('NOX', 'RM', 'LSTAT')},
 4: {'feature_idx': (3, 4, 5, 12),
  'cv_scores': array([0.93700787, 0.95275591, 0.91269841, 0.93650794]),
  'avg_score': 0.934742532183477,
  'feature_names': ('CHAS', 'NOX', 'RM', 'LSTAT')}}

In [53]:
sfs1.k_feature_names_

('CHAS', 'NOX', 'RM', 'LSTAT')

In [54]:
sfs1.k_score_

0.934742532183477

[back to outline](#outline)